# QR Code Recognition using OpenCV 
**Full Codiig available at [QR_Code_Recognition.py](QR_Code_Recognition.py)**
<br>
This is the documantetion for QR Code Recognition 

***Librarues Used In The Script*** 
- Picamera2 
- libcamera 
- OpenCV 
- time 
- RPi_Robot_Hat_Lib

## Installing Libraries 
1. OpenCV [OpenCV.org](https://docs.opencv.org/4.x/d2/de6/tutorial_py_setup_in_ubuntu.html)
    - In the terminal type `pip install opencv-python` 
    - In a Python File type: <br>
    `import cv2` <br>
    `print(cv2.__version__)`
    - The coding above will print the version of opencv library installed  

## Let's Start Coding !  
### 1. Import the required libraries 
- OpenCV
- Picamera2 
- libcamera 
- time 
- RPi_Robot_Hat_Lib

In [ ]:
import cv2 
from picamera2 import Picamera2 
from libcamera import controls
import time 
from RPi_Robot_Hat_Lib import RobotController


### 2. Configure the camera 
- Initialise RobotController Library 
- Define channel for vertical and horizontal sevo PanTilt HAT 
- Set the position of the camera (Using PanTilt HAT)
- Width: 640 
- height: 480 
- Autofocus: ON **(This function is only available for Raspberry Pi camera Module 3 and above)**

In [ ]:
Motor = RobotController()
vertical = 2
horizontal = 1
Motor.servoPulse(horizontal, 80)
Motor.servoPulse(vertical, 90)

frame_width = 640 
frame_height = 480 
cam = Picamera2()
cam.configure(cam.create_preview_configuration(main={"format": 'XRGB8888', "size": (frame_width, frame_height)})) 
cam.start()
cam.set_controls({"AfMode":controls.AfModeEnum.Continuous})

### 3. Initialise the detector 
- Initialise the detection module form OpenCV library 

In [ ]:
detector = cv2.QRCodeDetector()

### 4. Initialise the variables for FPS (Frame Per Second) counter 
- Get the start time from the time library 
- Initialise the FPS variable to 0. 

In [ ]:
t_start = time.time()
fps = 0

### 5.The While True Loop 
- The while true loop holsd the codes that will loop forever when the programe starts.
- Obtain the capture image from the camera using the Picamera2 Library. 
    ```py
    frame = cam.capture_array()
    ```
- Extract the data of the QR Code from the capture image 
- Extract the coordinates of the QR Code Detected.
    ```py
    data, bbox, _ = detector.detectAndDecode(frame)
    ```
- Calculate the FPS of the camera stream 
    ```py
    fps +=1
    mfps = fps/(time.time() - t_start)
    ``` 
- If the QR Code is detected,  draw a bounding box around the QR Code. 
- Display the data extracted from the QR Code. 
    ```py
    if(bbox is not None):
        for i in range(len(bbox)):
            
            pt1 = (int(bbox[i][0][0]), int(bbox[i][0][1]))
            pt2 = (int(bbox[(i + 1) % len(bbox)][0][0]), int(bbox[(i + 1) % len(bbox)][0][1]))
            
            cv2.line(frame, pt1, pt2, color=(255,
                     0, 0), thickness=2)
            
        cv2.putText(frame, data, (int(bbox[0][0][0]), int(bbox[0][0][1]) - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (255, 250, 120), 2)
    else:
        print("No Data Found")
    ```
- Display the FPS calculated for the camera stream 
- Display the camera stream will all the anotation on a window. 
    ```py
    cv2.putText(frame, "FPS : " + str(int(mfps)), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.imshow("code detector", frame)
    ```
- If the button **q** is pressed, close the program 
    ```py
     if cv2.waitKey(1) == ord("q"):
        break
    ```

In [ ]:
while True :

    frame = cam.capture_array()
    data, bbox, _ = detector.detectAndDecode(frame)
    fps +=1
    mfps = fps/(time.time() - t_start)

    if(bbox is not None):
        for i in range(len(bbox)):
            
            pt1 = (int(bbox[i][0][0]), int(bbox[i][0][1]))
            pt2 = (int(bbox[(i + 1) % len(bbox)][0][0]), int(bbox[(i + 1) % len(bbox)][0][1]))
            
            cv2.line(frame, pt1, pt2, color=(255,
                     0, 0), thickness=2)
            
        cv2.putText(frame, data, (int(bbox[0][0][0]), int(bbox[0][0][1]) - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (255, 250, 120), 2)

        
        
    else:
        print("No Data Found")

    cv2.putText(frame, "FPS : " + str(int(mfps)), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    cv2.imshow("code detector", frame)

    if cv2.waitKey(1) == ord("q"):
        break 

### 6. Wrapping things up  
- Stop the camera 
- Reset the servo positon to default 
- using OpenCV to destroy all the windows oopened by the opencv when the progrom runs. 

In [ ]:
cam.stop()
Motor.cleanup()
cv2.destroyAllWindows() 